This is the setup cell

In [1]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt

#Run this line if you want interactive plots
%matplotlib qt

import numpy as np

import sprit

In [2]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\ChampaignCo\Mahomet2023\Data\RawData"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\RAC84"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\UOFI1_AM.RAC84.00.2023-02-15_1704-1734.mseed"

params = sprit.input_param( acq_date='2023-02-15',
                            #starttime = obspy.UTCDateTime('2023-02-06T21:32:00.00'),
                            #endtime = obspy.UTCDateTime('2023-02-06T22:00:00.00'),
                            tzone = 'UTC',#or 'local', #or 'UTC'
                            dst=False,
                            lon = -88.2290526,
                            lat =  40.1012122,
                            elevation = 755,
                            site = 'HVSR Site',
                            dataPath = dPath
                            )

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/riley/LocalData/Github/SPRIT/resources/raspshake_metadata.inv


In [3]:
params = sprit.get_metadata(params)
stream = sprit.fetch_data(params=params)

Day of Year: 46


c:\ProgramData\Anaconda3\envs\seismic39\lib\site-packages\obspy\core\inventory\network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


In [4]:
xwindows = sprit.select_windows(stream)

[[19403.7164385426, 19403.717423029608], [19403.72419137778, 19403.725914230043], [19403.719925267415, 19403.721196896466]]
19403.724888722743
1
[[19403.7164385426, 19403.717423029608], [19403.719925267415, 19403.721196896466]]
[[19403.7164385426, 19403.717423029608], [19403.719925267415, 19403.721196896466]]
19403.71664364406
0
[[19403.719925267415, 19403.721196896466]]
[[19403.719925267415, 19403.721196896466]]
19403.720909754422
0
[]
[[19403.715659157053, 19403.7174640499], [19403.72419137778, 19403.725709128583]]
19403.724929743035
1
[[19403.715659157053, 19403.7174640499]]


In [4]:
params = sprit.generate_ppsds(params=params, stream=stream, ppsd_length=60, overlap=0.5, period_step_octaves=0.01)
hvsr_dict = sprit.process_hvsr(params=params, method=4, resample=False, smooth=10)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

In [5]:
plt.ion()
sprit.hvplot(hvsr_dict, kind='HVSR', show=True)

In [16]:
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

#fig, ax = plt.subplots()


fig, ax = sprit.hvplot(hvsr_dict=hvsr_dict, kind='spec', returnfig=True, cmap='turbo')

clickNo = 0    
xWindows = []
pathList = []
windowDrawn = []
winArtist = []
lineArtist = []

def on_click(event, fig=fig, ax=ax):
    if event.button is MouseButton.RIGHT:
        remove_on_right(event)
    if event.button is MouseButton.LEFT:
        draw_boxes(event)

def draw_boxes(event, fig=fig, ax=ax):
    global clickNo
    global x0
    global xWindows
    global pathList
    global windowDrawn
    global winArtist
    global lineArtist

    if event.inaxes!=ax: return
    #y0, y1 = ax.get_ylim()
    y0=0
    y1=50
    
    if clickNo == 0:
        #y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 2)
        x0 = event.xdata
        clickNo += 1
        linArt = plt.axvline(x0, y0, y1, linewidth=0.5, color='k', zorder=100)
        lineArtist.append([linArt, linArt])
    else:
        x1 = event.xdata
        clickNo = 0

        path_data = [
            (matplotlib.path.Path.MOVETO, (x0, y0)),
            (matplotlib.path.Path.LINETO, (x1, y0)),
            (matplotlib.path.Path.LINETO, (x1, y1)),
            (matplotlib.path.Path.LINETO, (x0, y1)),
            (matplotlib.path.Path.LINETO, (x0, y0)),
            (matplotlib.path.Path.CLOSEPOLY, (x0, y0)),
        ]

        codes, verts = zip(*path_data)
        path = matplotlib.path.Path(verts, codes)

        x_win = [x0, x1]
        xWindows.append(x_win)
        pathList.append(path)
        windowDrawn.append(False)
        winArtist.append(None)
        [lineArtist[-1].pop()]
        draw_windows(event=event, pathlist=pathList)
        linArt = plt.axvline(x1, y0, y1, color='k',linewidth=0.5, zorder=100)
        lineArtist[-1].append(linArt)
    fig.canvas.draw() 

def draw_windows(event, pathlist, fig=fig, ax=ax):
    for i, p in enumerate(pathList):
        if windowDrawn[i]:
            pass
        else:
            patch = matplotlib.patches.PathPatch(p, facecolor='k', alpha=0.75)
            winArt = ax.add_patch(patch)
            windowDrawn[i] = True
            winArtist[i] = winArt
    if event.button is MouseButton.RIGHT:
        fig.canvas.draw() 

def remove_on_right(event, fig=fig, ax=ax, xWindows=xWindows):
    if xWindows is not None:
        for i, xWins in enumerate(xWindows):
            if event.xdata > xWins[0] and event.xdata < xWins[1]:
                xWindows.pop(i)
                pathList.pop(i)
                winArtist[i].remove()
                winArtist.pop(i)
                windowDrawn.pop(i)
                lineArtist[i][0].remove()
                lineArtist[i][1].remove()
                lineArtist.pop(i)
    fig.canvas.draw() 
    #draw_windows(event, pathlist=pathList)

fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()